### Please download the model from Hugging Face

### Hugging Face Model Repo:
https://huggingface.co/JesseHuang922/lv_roberta_large

Due to GitHub's file size limit (maximum 100MB per file), the larger RoBERTa-based models are not included in this repository.

Specifically, files under ./models/* and ./packages/* are excluded from version control.


In [1]:
# ==============================
# Cell 1：Import and directories
# ==============================
from pathlib import Path
import os
import spacy
from spacy.lookups import Lookups
from spacy.tokens import DocBin
from spacy.cli.package import package

# Project root dir
project_root = Path(".").resolve()

# Project structure
models_dir = project_root / "models"
model_name = "lv_roberta_large"
trained_model_path = models_dir / model_name / "model-best"
final_model_path = models_dir / model_name / "model_roberta_large"
lookups_path = project_root / "lookups_lv"
package_output_dir = project_root / "packages"
config_path = project_root / "config" / "config_roberta_large_20.cfg"

# Create directories
for p in [models_dir, models_dir / model_name, package_output_dir, lookups_path, project_root / "config"]:
    p.mkdir(parents=True, exist_ok=True)

print("Imports and directories are created")


Imports and directories are created


In [2]:
# =============================
# Cell 2：Initializing config
# =============================
!python -m spacy init config ./config/config_roberta_large_20.cfg \
    --lang lv \
    --pipeline transformer,tagger,morphologizer,parser,senter \
    --optimize efficiency \
    --gpu


ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser, senter
- Optimize for: efficiency
- Hardware: GPU
- Transformer: bert-base-multilingual-uncased
✔ Auto-filled config with all values
✔ Saved config
config/config_roberta_large_20.cfg
You can now add your data and train your pipeline:
python -m spacy train config_roberta_large_20.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [3]:
# ==========================
# Cell 3：Modify config
# ==========================

# corpus dir
subset = "20pct"  
corpus_dir = Path("/home/jesse/Projects/myprojs/Master_Thesis/Fengdi_Huang_Master_Thesis_Repo/Corpus/train")

# Read config file
cfg_text = config_path.read_text(encoding="utf-8")

# Replace the training/validation set path
cfg_text = cfg_text.replace("train = null", f"train = {corpus_dir}/lv_lvtb-ud-train-{subset}.spacy")
cfg_text = cfg_text.replace("dev = null", f"dev = {corpus_dir}/lv_lvtb-ud-dev.spacy")

# Change transformer model to xlm-roberta-large
cfg_text = cfg_text.replace("bert-base-multilingual-uncased", "xlm-roberta-large")

# Turn on Mixed Precision
cfg_text = cfg_text.replace("mixed_precision = false", "mixed_precision = true")

# Modify pipeline： add trf_tok2vec component to pipeline
cfg_text = cfg_text.replace(
    'pipeline = ["transformer","tagger","morphologizer","parser","senter"]',
    'pipeline = ["transformer","trf_tok2vec","tagger","morphologizer","parser","senter"]'
)

# Add trf_tok2vec component config
if "[components.trf_tok2vec]" not in cfg_text:
    trf_tok2vec_cfg = """
[components.trf_tok2vec]
factory = "tok2vec"

[components.trf_tok2vec.model]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"
"""
    cfg_text += trf_tok2vec_cfg

config_path.write_text(cfg_text, encoding="utf-8")
print("Config updated: training/calidation set path, transformer base model, mixed precision and pipeline + trf_tok2vec components are all set")


Config updated: training/calidation set path, transformer base model, mixed precision and pipeline + trf_tok2vec components are all set


In [4]:
# ===================================
# Cell 4：Generate lemma lookup table
# ===================================
from spacy.tokens import DocBin
from spacy.lookups import Lookups
import spacy
from pathlib import Path

# Paths
corpus_dir= Path("/home/jesse/Projects/myprojs/Master_Thesis/Fengdi_Huang_Master_Thesis_Repo/Corpus/train")
lookups_path = Path("lookups_lv")

# --------------- Choose generation mode ---------------
# Mode 1: Strict evaluation mode (train + dev only)
# files = ["lv_lvtb-ud-train.spacy", "lv_lvtb-ud-dev.spacy"]

# Mode 2: Practical enhanced mode (train + dev + test)
files = ["lv_lvtb-ud-train-20pct.spacy"]

# --------------- Generate lemma lookup ---------------
lemma_dict = {}
nlp_blank = spacy.blank("lv")

for file_name in files:
    docbin = DocBin().from_disk(corpus_dir / file_name)
    for doc in docbin.get_docs(nlp_blank.vocab):
        for token in doc:
            if token.lemma_:
                lemma_dict[token.text.lower()] = token.lemma_

lookups = Lookups()
lookups.add_table("lemma_lookup", lemma_dict)
lookups.to_disk(lookups_path)

print(f"✅ Lemma lookup table generated, mode: {files}, saved at: {lookups_path}")

/home/jesse/Projects/myenvs/master_thesis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Lemma lookup table generated, mode: ['lv_lvtb-ud-train-20pct.spacy'], saved at: lookups_lv


In [5]:
# ================================
# Cell 5：Train RoBERTa large model
# ================================
!python -m spacy train ./config/config_roberta_large_20.cfg \
    --output ./models/lv_roberta_large \
    --gpu-id 0


ℹ Saving to output directory: models/lv_roberta_large
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'trf_tok2vec', 'tagger', 'morphologizer',
'parser', 'senter']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TRF_T...  LOSS TAGGER  LOSS MORPH...  LOSS PARSER  LOSS SENTER  TAG_ACC  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  SCORE 
---  ------  -------------  -------------  -----------  -------------  -----------  -----------  -------  -------  ---------  -------  -------  -------  -------  -------  ------
/home/jesse/Projects/myenvs/master_thesis/lib/python3.12/site-packages/thinc/util.py:400: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  dlpack_tensor = xp_tensor.toDlpack()

In [6]:
# ========================
# Cell 6: Evaluation (parametric)
# ========================
model_path = f"./models/lv_roberta_large/model-best"
test_path = "/home/jesse/Projects/myprojs/Master_Thesis/Fengdi_Huang_Master_Thesis_Repo/Corpus/test/lv_lvtb-ud-test.spacy"

!python -m spacy evaluate {model_path} {test_path} --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK      99.53
TAG      88.90
POS      96.94
MORPH    93.71
UAS      89.00
LAS      84.76
SENT P   94.19
SENT R   97.47
SENT F   95.80
SPEED    9520 


============================== MORPH (per feat) ==============================

               P       R       F
ExtPos     92.03   90.71   91.37
Case       97.22   96.62   96.92
Gender     95.87   95.26   95.56
Number     97.07   96.52   96.79
Person     98.31   98.11   98.21
PronType   98.35   97.86   98.11
Evident    98.56   98.43   98.50
Mood       98.02   97.74   97.88
Polarity   98.81   98.60   98.70
Tense      95.68   95.59   95.64
VerbForm   98.29   98.08   98.18
Voice      98.26   98.03   98.14
Definite   95.08   96.18   95.63
Degree     97.56   97.79   97.67
Poss       99.45   97.85   98.64
NumType    96.47   73.43   83.39
Reflex     98.26   97.10   97.68
Aspect     96.93   96.33   96.63
Foreign    86.78   79.47   82.97
Typo       50

In [7]:
# =================================================================================
# Cell 7：Add Lemmatizer (lookup) to model + Copy LICENSE, LICENSE_SOURCES & README
# =================================================================================
import spacy
from spacy.lookups import Lookups
from pathlib import Path
import shutil

trained_model_path = "./models/lv_roberta_large/model-best"
final_model_path = Path("./models/lv_roberta_large/model_roberta_large")
lookups_path = "./lookups_lv"

# Load trained model
nlp = spacy.load(trained_model_path)

# Add lookups
lookups = Lookups().from_disk(lookups_path)

# Add lemmatizer to pipeline
lemmatizer = nlp.add_pipe("lemmatizer", config={"mode": "lookup"}, last=True)
lemmatizer.lookups = lookups  # assign properties directly here

# Save new model with lemmatizer components
nlp.to_disk(final_model_path)
print(f"✅ Model saved to: {final_model_path} with lemmatizer + lookups")


✅ Model saved to: models/lv_roberta_large/model_roberta_large with lemmatizer + lookups


In [8]:
# =======================
# Cell 8：Packaging
# =======================

from spacy.cli.package import package
from pathlib import Path
import os

project_root = Path(".").resolve()
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH','')}"

# Note that the string path is replaced with a Path object (don't know why but it works only this way)
package(
    input_dir=Path(final_model_path),
    output_dir=Path(package_output_dir),
    name="xlmr_large_20pct",
    version="1.0.0",
    force=True
)

print(f"Finished, packaged model can be found here: {package_output_dir}")

/home/jesse/Projects/myenvs/master_thesis/bin/python: No module named build


running sdist
running egg_info
writing lv_xlmr_large_20pct.egg-info/PKG-INFO
writing dependency_links to lv_xlmr_large_20pct.egg-info/dependency_links.txt
writing entry points to lv_xlmr_large_20pct.egg-info/entry_points.txt
writing requirements to lv_xlmr_large_20pct.egg-info/requires.txt
writing top-level names to lv_xlmr_large_20pct.egg-info/top_level.txt
reading manifest file 'lv_xlmr_large_20pct.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_xlmr_large_20pct.egg-info/SOURCES.txt'
running check
creating lv_xlmr_large_20pct-1.0.0
creating lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct
creating lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct.egg-info
creating lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0
creating lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/lemmatizer/lookups
creating lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/morphologizer
creating lv_xlmr_large_20

copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/trf_tok2vec/cfg -> lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/trf_tok2vec
copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/trf_tok2vec/model -> lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/trf_tok2vec
copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab/key2row -> lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab
copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab/lookups.bin -> lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab
copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab/strings.json -> lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab
copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab/vectors -> lv_xlmr_large_20pct-1.0.0/lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab
copying lv_xlmr_large_20pct/lv_xlmr_large_20pct-1.0.0/vocab/vectors.cfg -> lv_x

In [9]:
# ===============================
# Cell 9： Build wheel + sdist
# ===============================
import subprocess
from pathlib import Path

package_output_dir = Path("./packages/lv_xlmr_large_20pct-1.0.0")
dist_dir = package_output_dir / "dist"

print(f"✅ sdist is ready. You can install it with pip from: {dist_dir}")

✅ sdist is ready. You can install it with pip from: packages/lv_xlmr_large_20pct-1.0.0/dist


In [10]:
# ================================
# Cell 10B: Install with 'tar.gz'
# ================================
import subprocess
import spacy
import pandas as pd

# Install with '.tar.gz'
subprocess.run(["pip", "install", "./packages/lv_xlmr_large_20pct-1.0.0/dist/lv_xlmr_large_20pct-1.0.0.tar.gz"])
nlp_xlmr = spacy.load("lv_xlmr_large_20pct")

print("lv_xlmr_large Pipeline components:", nlp_xlmr.pipe_names)

Processing ./packages/lv_xlmr_large_20pct-1.0.0/dist/lv_xlmr_large_20pct-1.0.0.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for lv_xlmr_large_20pct: filename=lv_xlmr_large_20pct-1.0.0-py3-none-any.whl size=1821730988 sha256=22492be840ee383df5d1e6812b45bdc58beeb4a0a67b7f8378df7bb925edc047
  Stored in directory: /home/jesse/.cache/pip/wheels/57/ad/92/a190be5e08c94f9d01537ff91dfa4306a13d79ce9029b40654
Successfully built lv_xlmr_large_20pct
lv_xlmr_large Pipeline components: ['transformer', 'trf_tok2vec', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [11]:
# ==================
# Cell 11A: Demo Testing
# ==================

import spacy
import numpy as np

# Load the pipeline
nlp = spacy.load("lv_xlmr_large_20pct")

# Example text
text = """Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm.
Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā."""

# Process text
doc = nlp(text)

# ---------------
# Tokenization 
# ---------------
print("Tokens: ")
print([token.text for token in doc])

# ---------------
# Lemmatization 
# ---------------
print("Lemmas: ")
print([token.lemma_ for token in doc])

# ------------------------
# Part-of-Speech Tagging 
# ------------------------
print("POS tags:")
for token in doc:
    print(f"{token.text}: {token.pos_} ({token.tag_})")

# ------------------------
# Morphological Features
# ------------------------
print("Morphological features:")
for token in doc:
    print(f"{token.text}: {token.morph}")

# ------------------------
# Dependency Parsing 
# ------------------------
print("Dependency parsing:")
for token in doc:
    print(f"{token.text} <--{token.dep_}-- {token.head.text}")

# ------------------------
# Sentence Segmentation 
# ------------------------
print("Sentences:")
for sent in doc.sents:
    print(sent.text)

# ------------------------
# Print pipeline components
# ------------------------
print("Pipeline components: ")
print(nlp.pipe_names)

# Tok2Vec
vectors = np.vstack([token.vector for token in doc])
print("Token vectors shape / Token:", vectors.shape)

Tokens: 
['Baltijas', 'jūras', 'nosaukums', 'ir', 'devis', 'nosaukumu', 'baltu', 'valodām', 'un', 'Baltijas', 'valstīm', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietoja', 'vācu', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimtā', '.']
Lemmas: 
['Baltijas', 'jūra', 'nosaukums', 'būt', 'devis', 'nosaukumu', 'balts', 'valodām', 'un', 'Baltijas', 'valstīm', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietoja', 'vāci', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimtā', '.']
POS tags:
Baltijas: PROPN (npfsg4)
jūras: NOUN (ncfsg4)
nosaukums: NOUN (ncmsn1)
ir: AUX (vcnipii30an)
devis: VERB (vmnpdmsnasnpn)
nosaukumu: NOUN (ncmsa1)
baltu: ADJ (affpgnp)
valodām: NOUN (ncfpd4)
un: CCONJ (cc)
Baltijas: PROPN (npfsg4)
valstīm: NOUN (ncfpd5)
.: PUNCT (zs)

: PUNCT (zs)
Terminu: NOUN (ncmsa1)
": PUNCT (zq)
Baltijas: PROPN (npfsg4)
jūra: NOUN (ncfsn4)
": PUNCT (zq)
(: PUNCT (zb)
Mare: X

In [12]:
# ===================================================
# Cell 11B: Testing model, espically Lemma and senter
# ===================================================
import spacy
import pandas as pd

nlp = spacy.load("lv_xlmr_large_20pct")

text = """Rīga ir Latvijas galvaspilsēta un viens no galvenajiem rūpniecības, darījumu, kultūras, sporta un finanšu centriem Baltijas valstīs, kā arī nozīmīga ostas pilsēta. Ar 605 273 iedzīvotājiem (2024. gada dati) tā ir lielākā apdzīvotā vieta Latvijā. Tās robežās dzīvo aptuveni viena trešdaļa, bet Rīgas aglomerācijā — vairāk nekā puse visu Latvijas iedzīvotāju. Pilsētas teritorijas platība ir 307,17 km2. Rīgas vēsturiskais centrs ir iekļauts UNESCO Pasaules kultūras mantojuma sarakstā un ir ievērojams ar jūgendstila arhitektūru, kurai, pēc UNESCO viedokļa, nav līdzīgu pasaulē. Kopš dibināšanas 1201. gadā līdz mūsu dienām Rīga ir Baltijas valstu lielākā pilsēta un viena no ievērojamākajām ostām Baltijas jūras austrumdaļā. Politiski un administratīvi tā ilgu laiku bijusi reģiona politiskais centrs, bet sākot ar 20. gadsimtu — Latvijas Republikas galvaspilsēta."""

doc = nlp(text)

# Generate Token Table
rows = []
for sent in doc.sents:
    for token in sent:
        rows.append({
            "Text": token.text,
            "Lemma": token.lemma_,
            "POS": token.pos_,
            "Dependency": token.dep_,
            "Head": token.head.text
        })

df = pd.DataFrame(rows)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
display(df) 

# Sentence Segmentation
print("\nSentence Segmentation results:")
for i, sent in enumerate(doc.sents, 1):
    print(f"Sentence {i}: {sent.text}")

,Text,Lemma,POS,Dependency,Head
0,Rīga,Rīga,PROPN,nsubj,galvaspilsēta
1,ir,būt,AUX,cop,galvaspilsēta
2,Latvijas,Latvijas,PROPN,nmod,galvaspilsēta
3,galvaspilsēta,galvaspilsēta,NOUN,ROOT,galvaspilsēta
4,un,un,CCONJ,cc,viens
5,viens,viens,NUM,conj,galvaspilsēta
6,no,no,ADP,case,centriem
7,galvenajiem,galvenajiem,ADJ,amod,centriem
8,rūpniecības,rūpniecības,NOUN,nmod,centriem
9,",",",",PUNCT,punct,darījumu



Sentence Segmentation results:
Sentence 1: Rīga ir Latvijas galvaspilsēta un viens no galvenajiem rūpniecības, darījumu, kultūras, sporta un finanšu centriem Baltijas valstīs, kā arī nozīmīga ostas pilsēta.
Sentence 2: Ar 605 273 iedzīvotājiem (2024. gada dati) tā ir lielākā apdzīvotā vieta Latvijā.
Sentence 3: Tās robežās dzīvo aptuveni viena trešdaļa, bet Rīgas aglomerācijā — vairāk nekā puse visu Latvijas iedzīvotāju.
Sentence 4: Pilsētas teritorijas platība ir 307,17 km2.
Sentence 5: Rīgas vēsturiskais centrs ir iekļauts UNESCO Pasaules kultūras mantojuma sarakstā un ir ievērojams ar jūgendstila arhitektūru, kurai, pēc UNESCO viedokļa, nav līdzīgu pasaulē.
Sentence 6: Kopš dibināšanas 1201. gadā līdz mūsu dienām Rīga ir Baltijas valstu lielākā pilsēta un viena no ievērojamākajām ostām Baltijas jūras austrumdaļā.
Sentence 7: Politiski un administratīvi tā ilgu laiku bijusi reģiona politiskais centrs, bet sākot ar 20. gadsimtu — Latvijas Republikas galvaspilsēta.


In [13]:
import spacy
from spacy.tokens import DocBin

# === 1. Set file paths ===
train_path = "/home/jesse/Projects/myprojs/Master_Thesis/Fengdi_Huang_Master_Thesis_Repo/Corpus/train/lv_lvtb-ud-train-20pct.spacy"
test_path  = "/home/jesse/Projects/myprojs/Master_Thesis/Fengdi_Huang_Master_Thesis_Repo/Corpus/test/lv_lvtb-ud-test.spacy"

# === 2. Load the model ===
nlp = spacy.load("lv_xlmr_large_20pct")

# === 3. Load the training data and build the vocabulary ===
train_vocab = set()
train_docs = DocBin().from_disk(train_path)
for doc in train_docs.get_docs(nlp.vocab):
    for token in doc:
        train_vocab.add(token.text)

print(f"✅ Training vocabulary size: {len(train_vocab)}")

# === 4. Load the test data ===
test_docs = DocBin().from_disk(test_path)
test_docs = list(test_docs.get_docs(nlp.vocab))
print(f"✅ Number of test documents: {len(test_docs)}")

# === 5. Initialize counters for all components ===
metrics = {
    "POS": {"iv_total": 0, "iv_correct": 0, "oov_total": 0, "oov_correct": 0},
    "MORPH": {"iv_total": 0, "iv_correct": 0, "oov_total": 0, "oov_correct": 0},
    "LEMMA": {"iv_total": 0, "iv_correct": 0, "oov_total": 0, "oov_correct": 0},
    "UAS": {"iv_total": 0, "iv_correct": 0, "oov_total": 0, "oov_correct": 0},
    "LAS": {"iv_total": 0, "iv_correct": 0, "oov_total": 0, "oov_correct": 0},
}

# === 6. Run predictions and calculate metrics ===
for gold_doc in test_docs:
    pred_doc = nlp(gold_doc.text)
    for gold_token, pred_token in zip(gold_doc, pred_doc):
        is_iv = gold_token.text in train_vocab

        # POS accuracy
        if is_iv:
            metrics["POS"]["iv_total"] += 1
            if gold_token.pos_ == pred_token.pos_:
                metrics["POS"]["iv_correct"] += 1
        else:
            metrics["POS"]["oov_total"] += 1
            if gold_token.pos_ == pred_token.pos_:
                metrics["POS"]["oov_correct"] += 1

        # Morphology accuracy (exact match)
        gold_morph = gold_token.morph.to_dict()
        pred_morph = pred_token.morph.to_dict()
        if is_iv:
            metrics["MORPH"]["iv_total"] += 1
            if gold_morph == pred_morph:
                metrics["MORPH"]["iv_correct"] += 1
        else:
            metrics["MORPH"]["oov_total"] += 1
            if gold_morph == pred_morph:
                metrics["MORPH"]["oov_correct"] += 1

        # Lemmatization accuracy
        if is_iv:
            metrics["LEMMA"]["iv_total"] += 1
            if gold_token.lemma_ == pred_token.lemma_:
                metrics["LEMMA"]["iv_correct"] += 1
        else:
            metrics["LEMMA"]["oov_total"] += 1
            if gold_token.lemma_ == pred_token.lemma_:
                metrics["LEMMA"]["oov_correct"] += 1

        # Parsing: UAS / LAS
        if gold_token.head is not None:
            if is_iv:
                metrics["UAS"]["iv_total"] += 1
                metrics["LAS"]["iv_total"] += 1
                if gold_token.head.i == pred_token.head.i:
                    metrics["UAS"]["iv_correct"] += 1
                    if gold_token.dep_ == pred_token.dep_:
                        metrics["LAS"]["iv_correct"] += 1
            else:
                metrics["UAS"]["oov_total"] += 1
                metrics["LAS"]["oov_total"] += 1
                if gold_token.head.i == pred_token.head.i:
                    metrics["UAS"]["oov_correct"] += 1
                    if gold_token.dep_ == pred_token.dep_:
                        metrics["LAS"]["oov_correct"] += 1

# === 7. Print the results ===
print("\n🎯 OOV/IV Accuracy Results")
print("--------------------------------------------------")
for comp, m in metrics.items():
    iv_acc = m["iv_correct"] / m["iv_total"] * 100 if m["iv_total"] > 0 else 0
    oov_acc = m["oov_correct"] / m["oov_total"] * 100 if m["oov_total"] > 0 else 0
    print(f"{comp}:")
    print(f"  IV  Accuracy  = {iv_acc:.2f}%  ({m['iv_correct']}/{m['iv_total']})")
    print(f"  OOV Accuracy = {oov_acc:.2f}%  ({m['oov_correct']}/{m['oov_total']})")
    print("--------------------------------------------------")


✅ Training vocabulary size: 12123
✅ Number of test documents: 2412

🎯 OOV/IV Accuracy Results
--------------------------------------------------
POS:
  IV  Accuracy  = 93.27%  (22717/24356)
  OOV Accuracy = 88.21%  (11277/12784)
--------------------------------------------------
MORPH:
  IV  Accuracy  = 92.06%  (22423/24356)
  OOV Accuracy = 80.33%  (10269/12784)
--------------------------------------------------
LEMMA:
  IV  Accuracy  = 85.63%  (20855/24356)
  OOV Accuracy = 20.39%  (2607/12784)
--------------------------------------------------
UAS:
  IV  Accuracy  = 85.50%  (20825/24356)
  OOV Accuracy = 80.88%  (10340/12784)
--------------------------------------------------
LAS:
  IV  Accuracy  = 82.13%  (20004/24356)
  OOV Accuracy = 75.69%  (9676/12784)
--------------------------------------------------
